In [1]:
import requests
import json
import time
from datetime import datetime
import pandas as pd

# Replace these variables with your Zabbix server details and credentials
ZABBIX_URL = 'http://10.100.116.112/api_jsonrpc.php'
ZABBIX_USER = 'Admin'
ZABBIX_PASSWORD = 'Bcs@12345$#$'
# HOSTNAME = 'Janapriya_Win_10.2.20.2'
TIME_PERIOD = 3600  # Time period in seconds

# Function to make API calls
def zabbix_api(method, params):
    headers = {'Content-Type': 'application/json-rpc'}
    data = {
        'jsonrpc': '2.0',
        'method': method,
        'params': params,
        'auth': auth_token,
        'id': 1
    }
    response = requests.post(ZABBIX_URL, headers=headers, data=json.dumps(data))
    return response.json()

# Step 1: Authenticate and get the auth token
auth_params = {
    'username': ZABBIX_USER,
    'password': ZABBIX_PASSWORD
}
auth_data = {
    'jsonrpc': '2.0',
    'method': 'user.login',
    'params': auth_params,
    'id': 1
}
auth_response = requests.post(ZABBIX_URL, headers={'Content-Type': 'application/json-rpc'}, data=json.dumps(auth_data))
# print(auth_response.json())
auth_token = auth_response.json().get('result')
auth_token

'c4f115d435fbabfaff3b4bb5179c3c03'

In [2]:
HOST_GROUP_NAME = 'mahabeer'
# Step 2: Get the host group ID
host_group_params = {
    'filter': {'name': [HOST_GROUP_NAME]},
    'output': ['groupid']
}
host_group_response = zabbix_api('hostgroup.get', host_group_params)
host_group_id = host_group_response['result'][0]['groupid']

# Step 3: Get hosts in the specified host group
host_params = {
    'groupids': host_group_id,
    'output': ['hostid', 'host'],
    'selectInterfaces': ['interfaceid', 'ip']
}
host_response = zabbix_api('host.get', host_params)
host_response

{'jsonrpc': '2.0',
 'result': [{'hostid': '10690',
   'host': 'Mahabeer Financial Services_Win_10.0.28.2',
   'interfaces': [{'interfaceid': '92', 'ip': '10.0.28.2'}]},
  {'hostid': '10691',
   'host': 'Mahabeer Financial Services_Win_10.0.28.5',
   'interfaces': [{'interfaceid': '93', 'ip': '10.0.28.5'}]},
  {'hostid': '10700',
   'host': 'Focus Softnet_Win_10.2.22.100',
   'interfaces': [{'interfaceid': '102', 'ip': '10.2.22.100'}]},
  {'hostid': '10710',
   'host': 'Kotamarketing_win_10.2.72.100',
   'interfaces': [{'interfaceid': '109', 'ip': '10.2.72.100'}]},
  {'hostid': '10692',
   'host': 'Janapriya_Win_10.2.20.2',
   'interfaces': [{'interfaceid': '94', 'ip': '10.2.20.2'}]}],
 'id': 1}

In [3]:
hostsAndIPs = {}
for host in host_response['result']:
    hostsAndIPs[host['hostid']] = host['interfaces'][0]['ip']
host_df = pd.DataFrame(list(hostsAndIPs.items()), columns=['hostid', 'ip_address'])
host_df["host_name"] = host_df["hostid"].apply(lambda x: [host['host'] for host in host_response['result'] if host['hostid'] == x][0])
host_df

,hostid,ip_address,host_name
0,10690,10.0.28.2,Mahabeer Financial Services_Win_10.0.28.2
1,10691,10.0.28.5,Mahabeer Financial Services_Win_10.0.28.5
2,10700,10.2.22.100,Focus Softnet_Win_10.2.22.100
3,10710,10.2.72.100,Kotamarketing_win_10.2.72.100
4,10692,10.2.20.2,Janapriya_Win_10.2.20.2


In [4]:
cpu_params = {
    "output": "extend",
        "hostids": list(hostsAndIPs.keys()),
        "with_triggers": True,
        "search": {
            "key_": "system.cpu"
        },
        "sortfield": "name"
}
mem_params = {
    "output": "extend",
        "hostids": list(hostsAndIPs.keys()),
        "with_triggers": True,
        "search": {
            "key_": "vm.memory.util"
        },
        "sortfield": "name"
}
cpu_response = zabbix_api('item.get', cpu_params)
# itemid_hostid_dict = {item['itemid']: item['hostid'] for item in cpu_response["result"]}
mem_response = zabbix_api('item.get', mem_params)
cpu_df = pd.DataFrame(cpu_response['result'], columns=['itemid', 'name', 'key_', 'hostid'])
cpu_df['ip'] = cpu_df['hostid'].apply(lambda x: hostsAndIPs[x])
cpu_df

,itemid,name,key_,hostid,ip
0,51318,CPU utilization,system.cpu.util,10690,10.0.28.2
1,51377,CPU utilization,system.cpu.util,10691,10.0.28.5
2,51466,CPU utilization,system.cpu.util,10692,10.2.20.2
3,51959,CPU utilization,system.cpu.util,10700,10.2.22.100
4,52327,CPU utilization,system.cpu.util,10710,10.2.72.100


In [28]:
def disk_params(disk_drive):
    return {
        "output": "extend",
        "hostids": list(hostsAndIPs.keys()),
        "with_triggers": True,
        "search": {
            "key_": f'vfs.fs.size[{disk_drive}:,pused]'
        },
        "sortfield": "name"
    }

disk_response_c = zabbix_api("item.get", disk_params("C"))
disk_response_d= zabbix_api("item.get", disk_params("D"))
disk_df_c = pd.DataFrame(disk_response_c['result'], columns=['itemid', 'name', 'key_', 'hostid'])
disk_df_d = pd.DataFrame(disk_response_d['result'], columns=['itemid', 'name', 'key_', 'hostid'])
disk_df_c['ip'] = disk_df_c['hostid'].apply(lambda x: hostsAndIPs[x])
disk_df_d['ip'] = disk_df_d['hostid'].apply(lambda x: hostsAndIPs[x])
disk_df_d

,itemid,name,key_,hostid,ip
0,51425,C:: Space utilization,"vfs.fs.size[C:,pused]",10690,10.0.28.2
1,51440,C:: Space utilization,"vfs.fs.size[C:,pused]",10691,10.0.28.5
2,51650,C:: Space utilization,"vfs.fs.size[C:,pused]",10692,10.2.20.2
3,52007,C:: Space utilization,"vfs.fs.size[C:,pused]",10700,10.2.22.100
4,52375,C:: Space utilization,"vfs.fs.size[C:,pused]",10710,10.2.72.100


In [29]:
disk_df_d

,itemid,name,key_,hostid,ip
0,51426,D:: Space utilization,"vfs.fs.size[D:,pused]",10690,10.0.28.2
1,51441,D:: Space utilization,"vfs.fs.size[D:,pused]",10691,10.0.28.5
2,51651,D:: Space utilization,"vfs.fs.size[D:,pused]",10692,10.2.20.2
3,52008,D:: Space utilization,"vfs.fs.size[D:,pused]",10700,10.2.22.100
4,52376,D:: Space utilization,"vfs.fs.size[D:,pused]",10710,10.2.72.100


In [6]:
mem_df = pd.DataFrame(mem_response['result'], columns=['itemid', 'name', 'key_', 'hostid'])
mem_df['ip'] = mem_df['hostid'].apply(lambda x: hostsAndIPs[x])
mem_df

,itemid,name,key_,hostid,ip
0,51328,Memory utilization,vm.memory.util,10690,10.0.28.2
1,51387,Memory utilization,vm.memory.util,10691,10.0.28.5
2,51476,Memory utilization,vm.memory.util,10692,10.2.20.2
3,51969,Memory utilization,vm.memory.util,10700,10.2.22.100
4,52337,Memory utilization,vm.memory.util,10710,10.2.72.100


In [7]:
cpu_history_params = {
    'output': 'extend',
    'history': 0,
    'itemids': [item['itemid'] for item in cpu_response['result']],
    'sortfield': 'clock',
    'sortorder': 'DESC',
    "time_from": int(datetime.now().timestamp()) - 7200,
    'time_till': int(datetime.now().timestamp())
}
cpu_history_response = zabbix_api('history.get', cpu_history_params)
cpu_hist_df = pd.DataFrame(cpu_history_response["result"])
cpu_hist_df

,itemid,clock,value,ns
0,51377,1717056677,2.414022,944580700
1,51959,1717056659,0.510411,355660300
2,51318,1717056618,6.697323,429630700
3,51466,1717056466,1.835417,428400000
4,52327,1717056427,8.865964,854200900
...,...,...,...,...
114,52327,1717049827,12.039115,323634800
115,51377,1717049777,33.76131,455481400
116,51959,1717049759,0.497379,234118900
117,51318,1717049718,4.929597,281080800


In [31]:
diskC_history_params = {
    'output': 'extend',
    'history': 0,
    'itemids': [item['itemid'] for item in disk_response_c['result']],
    'sortfield': 'clock',
    'sortorder': 'DESC',
    "time_from": int(datetime.now().timestamp()) - 7200,
    'time_till': int(datetime.now().timestamp())
}

diskC_history_response = zabbix_api('history.get', diskC_history_params)
diskC_hist_df = pd.DataFrame(diskC_history_response["result"])
diskC_hist_df

,itemid,clock,value,ns
0,52375,1717141975,80.239975,741239000
1,51440,1717141940,26.364458,782519900
2,51425,1717141925,13.633344,424593700
3,52007,1717141907,62.965102,312098300
4,51650,1717141850,58.563456,83182800
...,...,...,...,...
114,51650,1717135250,58.514689,270822200
115,52375,1717135075,80.180568,711951900
116,51440,1717135040,26.362898,69441100
117,51425,1717135025,13.631146,106904100


In [32]:
diskD_history_params = {
    'output': 'extend',
    'history': 0,
    'itemids': [item['itemid'] for item in disk_response_d['result']],
    'sortfield': 'clock',
    'sortorder': 'DESC',
    "time_from": int(datetime.now().timestamp()) - 7200,
    'time_till': int(datetime.now().timestamp())
}

diskD_history_response = zabbix_api('history.get', diskD_history_params)
diskD_hist_df = pd.DataFrame(diskD_history_response["result"])
diskD_hist_df

,itemid,clock,value,ns
0,51651,1717142151,58.009542,905532000
1,52376,1717141976,1.228412,802058600
2,51441,1717141941,83.18665,809880000
3,51426,1717141926,96.38886,737323700
4,52008,1717141908,99.999232,406573100
...,...,...,...,...
113,51426,1717135326,96.388733,50024800
114,52008,1717135308,99.99911,347651000
115,51651,1717135251,58.009697,494914200
116,52376,1717135076,1.226497,726278000


In [8]:
mem_history_params = {
    'output': 'extend',
    'history': 0,
    'itemids': [item['itemid'] for item in mem_response['result']],
    'sortfield': 'clock',
    'sortorder': 'DESC',
    "time_from": int(datetime.now().timestamp()) - 7200,
    'time_till': int(datetime.now().timestamp())
}
mem_history_response = zabbix_api('history.get', mem_history_params)
mem_hist_df = pd.DataFrame(mem_history_response["result"])
mem_hist_df

,itemid,clock,value,ns
0,51328,1717056628,23.779610144362472,175204000
1,51476,1717056476,24.06074450445665,29053800
2,52337,1717056437,56.20847478226084,1807700
3,51387,1717056387,70.16294026834836,284280100
4,51969,1717056369,63.92061333717011,265131700
...,...,...,...,...
113,51387,1717049787,72.66350354956576,585321400
114,51969,1717049769,63.908347261309494,565339800
115,51328,1717049728,24.432189326832134,526954200
116,51476,1717049576,24.0146276816095,386952500


In [9]:
cpu_hist_df['value'] = pd.to_numeric(cpu_hist_df['value'], errors='coerce')
cpu_mean_df = cpu_hist_df.groupby('itemid')['value'].mean()
cpu_max_df = cpu_hist_df.groupby('itemid')['value'].max()
cpu_max_df
# Convert Series to DataFrame
cpu_mean_df = cpu_mean_df.to_frame('cpu_mean')
cpu_max_df = cpu_max_df.to_frame('cpu_max')

# Reset index to make 'itemid' a column
cpu_mean_df.reset_index(level=0, inplace=True)
cpu_max_df.reset_index(level=0, inplace=True)

# Merge DataFrames
cpu_merged_df = pd.merge(cpu_mean_df, cpu_max_df, on='itemid')
cpu_merged_df = pd.merge(cpu_merged_df, cpu_df, on='itemid')
cpu_merged_df

,itemid,cpu_mean,cpu_max,name,key_,hostid,ip
0,51318,4.313168,16.185692,CPU utilization,system.cpu.util,10690,10.0.28.2
1,51377,34.491492,78.413879,CPU utilization,system.cpu.util,10691,10.0.28.5
2,51466,2.009951,2.291126,CPU utilization,system.cpu.util,10692,10.2.20.2
3,51959,0.535455,0.600944,CPU utilization,system.cpu.util,10700,10.2.22.100
4,52327,12.281333,52.696838,CPU utilization,system.cpu.util,10710,10.2.72.100


In [10]:
mem_hist_df['value'] = pd.to_numeric(mem_hist_df['value'], errors='coerce')
mem_mean_df = mem_hist_df.groupby('itemid')['value'].mean()
mem_max_df = mem_hist_df.groupby('itemid')['value'].max()
# Convert Series to DataFrame
mem_mean_df = mem_mean_df.to_frame('mem_mean')
mem_max_df = mem_max_df.to_frame('mem_max')

# Reset index to make 'itemid' a column
mem_mean_df.reset_index(level=0, inplace=True)
mem_max_df.reset_index(level=0, inplace=True)

# Merge DataFrames
mem_merged_df = pd.merge(mem_mean_df, mem_max_df, on='itemid')
mem_merged_df = pd.merge(mem_merged_df, mem_df, on='itemid')
mem_merged_df

,itemid,mem_mean,mem_max,name,key_,hostid,ip
0,51328,24.024076,24.432189,Memory utilization,vm.memory.util,10690,10.0.28.2
1,51387,68.549060,72.663504,Memory utilization,vm.memory.util,10691,10.0.28.5
2,51476,24.028875,24.060774,Memory utilization,vm.memory.util,10692,10.2.20.2
3,51969,63.915523,63.932718,Memory utilization,vm.memory.util,10700,10.2.22.100
4,52337,46.834306,56.208475,Memory utilization,vm.memory.util,10710,10.2.72.100


In [33]:
diskC_hist_df['value'] = pd.to_numeric(diskC_hist_df['value'], errors='coerce')
diskC_mean_df = diskC_hist_df.groupby('itemid')['value'].mean()
diskC_max_df = diskC_hist_df.groupby('itemid')['value'].max()
# Convert Series to DataFrame
diskC_mean_df = diskC_mean_df.to_frame('diskC_mean')
diskC_max_df = diskC_max_df.to_frame('diskC_max')
diskC_mean_df.reset_index(level=0, inplace=True)
diskC_max_df.reset_index(level=0, inplace=True)
diskC_merged_df = pd.merge(diskC_mean_df, diskC_max_df, on='itemid')
diskC_merged_df = pd.merge(diskC_merged_df, disk_df_c, on='itemid')
diskC_merged_df


,itemid,diskC_mean,diskC_max,name,key_,hostid,ip
0,51425,13.631981,13.633344,C:: Space utilization,"vfs.fs.size[C:,pused]",10690,10.0.28.2
1,51440,26.363708,26.364458,C:: Space utilization,"vfs.fs.size[C:,pused]",10691,10.0.28.5
2,51650,58.546791,58.578198,C:: Space utilization,"vfs.fs.size[C:,pused]",10692,10.2.20.2
3,52007,62.961058,62.965102,C:: Space utilization,"vfs.fs.size[C:,pused]",10700,10.2.22.100
4,52375,80.202585,80.276301,C:: Space utilization,"vfs.fs.size[C:,pused]",10710,10.2.72.100


In [35]:
diskD_hist_df['value'] = pd.to_numeric(diskC_hist_df['value'], errors='coerce')
diskD_mean_df = diskD_hist_df.groupby('itemid')['value'].mean()
diskD_max_df = diskD_hist_df.groupby('itemid')['value'].max()
# Convert Series to DataFrame
diskD_mean_df = diskD_mean_df.to_frame('diskD_mean')
diskD_max_df = diskD_max_df.to_frame('diskD_max')
diskD_mean_df.reset_index(level=0, inplace=True)
diskD_max_df.reset_index(level=0, inplace=True)
diskD_merged_df = pd.merge(diskD_mean_df, diskD_max_df, on='itemid')
diskD_merged_df = pd.merge(diskD_merged_df, disk_df_d, on='itemid')
diskD_merged_df


,itemid,diskD_mean,diskD_max,name,key_,hostid,ip
0,51426,62.961210,62.965102,D:: Space utilization,"vfs.fs.size[D:,pused]",10690,10.0.28.2
1,51441,13.631981,13.633344,D:: Space utilization,"vfs.fs.size[D:,pused]",10691,10.0.28.5
2,51651,80.202585,80.276301,D:: Space utilization,"vfs.fs.size[D:,pused]",10692,10.2.20.2
3,52008,58.546791,58.578198,D:: Space utilization,"vfs.fs.size[D:,pused]",10700,10.2.22.100
4,52376,26.363708,26.364458,D:: Space utilization,"vfs.fs.size[D:,pused]",10710,10.2.72.100


In [11]:
# Merge cpu and memory DataFrames
merged_df = pd.merge(cpu_merged_df, mem_merged_df, on='hostid', suffixes=('_cpu', '_mem'))

# Merge with host DataFrame
# host_df = pd.merge(host_df, merged_df, left_on='itemid', right_on='itemid')

# host_df
merged_df = merged_df[['hostid','ip_cpu', 'cpu_mean', 'cpu_max', 'mem_mean', 'mem_max']]
merged_df = merged_df.rename(columns={'ip_cpu': 'server', 'cpu_mean': 'cpu_avg', 'mem_mean' : 'memory_avg', 'mem_max' : 'memory_max'})
merged_df

,hostid,server,cpu_avg,cpu_max,memory_avg,memory_max
0,10690,10.0.28.2,4.313168,16.185692,24.024076,24.432189
1,10691,10.0.28.5,34.491492,78.413879,68.549060,72.663504
2,10692,10.2.20.2,2.009951,2.291126,24.028875,24.060774
3,10700,10.2.22.100,0.535455,0.600944,63.915523,63.932718
4,10710,10.2.72.100,12.281333,52.696838,46.834306,56.208475
